In [148]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch as torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models

In [149]:
import pickle as pkl
from tqdm.notebook import tqdm

In [150]:
DEVICE = torch.device("cuda:0")
DIM = 128
IMAGE_SIZE = 112
dtype = torch.cuda.FloatTensor

In [151]:
torch.cuda.is_available()

True

In [152]:
data_ = pkl.load(open('Data_siamese.pickle', 'rb'))

In [153]:
data_.shape

(6317, 20, 112, 112, 3)

In [168]:
data = data[:4000, :, :, :, :]

In [169]:
model = models.mobilenet_v2(width_mult=1, pretrained=True)
model = torch.nn.Sequential(*(list(model.children())[:-1]))

In [170]:
class Bottleneck(nn.Module):
    def __init__(self, model, in_dim=1280, out_dim=DIM, spartial=(IMAGE_SIZE+31) // 32):
        super().__init__()
        self.model = model
        self.depthwise = nn.Conv2d(in_dim, in_dim, spartial, bias=False, groups=in_dim)
        self.linear = nn.Linear(in_dim, out_dim)
        
    def forward(self, x):
        x = self.model(x)
        x = self.depthwise(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

In [171]:
model = Bottleneck(model)
model = model.to(DEVICE)

In [172]:
class Probcalc(nn.Module):
    def __init__(self, in_dim = 128):
        super().__init__()
        self.layer = nn.Linear(in_dim, 1)
    
    def forward(self, x):
        x = self.layer(x)
        x = torch.sigmoid(x)
        return x

In [173]:
calcprob = Probcalc().to(DEVICE)

In [174]:
optimizer = torch.optim.Adam(list(model.parameters()) + list(calcprob.parameters()), lr=0.003)

In [175]:
def change_lr(optimizer, epoch):
    for param_group in optimizer.param_groups:
        param_group['lr'] = 1e-3 if epoch < 30 else 1e-4

In [176]:
rng = np.random.default_rng()

In [177]:
import cv2
from PIL import Image

In [178]:
def dist(x1, x2):
    return torch.abs(x1 - x2)

In [179]:
EPOCHS = 60
STEP_PER_EPOCH = 2000
BATCH = 128

In [183]:
def gen_pair():
    batch_data_1 = torch.zeros(BATCH, 3, 112, 112).byte()
    batch_data_2 = torch.zeros(BATCH, 3, 112, 112).byte()
    batch_target = torch.zeros(BATCH, 1)
    bad = rng.choice(4000, size = BATCH, replace=False)
    good = rng.choice(4000, size = BATCH // 2, replace=True)
    for i in range(BATCH):
        if (i % 2 == 0):
            ind = rng.choice(20, size=2, replace=False)
            batch_data_1[i] = torch.tensor((data[good[i // 2], ind[0]] - 255 / 2) /  (255 / 2)).transpose(0, 2)
            batch_data_2[i] = torch.tensor((data[good[i // 2], ind[1]] - 255 / 2) /  (255 / 2)).transpose(0, 2)
            batch_target[i] = 1
        else:
            ind = rng.choice(20, size=2, replace=True)
            batch_data_1[i] = torch.tensor((data[bad[i // 2], ind[0]] - 255 / 2) /  (255 / 2)).transpose(0, 2)
            batch_data_2[i] = torch.tensor((data[bad[i // 2 + BATCH // 2], ind[1]] - 255 / 2) /  (255 / 2)).transpose(0, 2)
            batch_target[i, 0] = 0
    return batch_data_1, batch_data_2, batch_target

In [184]:
from torchvision import transforms

In [185]:
for epoch in range(EPOCHS):
    avg_loss = 0.0
    for s in tqdm(range(STEP_PER_EPOCH)):
        change_lr(optimizer, epoch)
        optimizer.zero_grad()        
        X1, X2, target = gen_pair()
        X1 = X1.to(DEVICE).float()
        X2 = X2.to(DEVICE).float()
        target = target.to(DEVICE).float()
        diff = dist(model(X1), model(X2))
        prob = calcprob(diff)
        loss = F.binary_cross_entropy(prob, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(list(model.parameters()) + list(calcprob.parameters()), 10)
        optimizer.step()
        avg_loss += loss.item()
        
    print("Train epoch", epoch, "finished with avg_loss", avg_loss / STEP_PER_EPOCH)
    avg_loss = 0
    avg_acc = 0


Train epoch 0 finished with avg_loss 0.6938010583519936



Train epoch 1 finished with avg_loss 0.6905699066817761



Train epoch 2 finished with avg_loss 0.6877190639972687



Train epoch 3 finished with avg_loss 0.6869825836122035



Train epoch 4 finished with avg_loss 0.6849676965773106


...


Train epoch 55 finished with avg_loss 0.6030126135647297



Train epoch 56 finished with avg_loss 0.603149858251214



Train epoch 57 finished with avg_loss 0.6013134385347366



Train epoch 58 finished with avg_loss 0.5992722234725952



Train epoch 59 finished with avg_loss 0.6004420094788074


In [186]:
torch.save(model.state_dict(), "model.state")
torch.save(calcprob.state_dict(), "dist2prob.state")